In [ ]:

import os
import zipfile
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns


# Step 1: Extract the dataset
dataset_zip_path = "/Users/zaivi/Desktop/Visual Basic/Robo Garden/archive (1).zip"
extract_path = "/Users/zaivi/Desktop/Visual Basic/Robo Garden/fruits_dataset"

# unzip
if not os.path.exists(extract_path):
    with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Dataset extracted successfully!")

# Define dataset directories
dataset_dir = os.path.join(extract_path, "fruits-360")
train_dir = os.path.join(dataset_dir, "Training")
test_dir = os.path.join(dataset_dir, "Test")

# Step 2: Preprocess Data
img_size = (100, 100)  
batch_size = 32

# Define ImageDataGenerator for preprocessing and augmentation
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.3)  # 70% Train, 30% Test

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

test_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

num_classes = len(train_generator.class_indices)  # Number of fruit categories

# Step 3: Define the CNN Model
model = keras.Sequential([
    # Input Layer
    keras.layers.Conv2D(16, (2, 2), activation='relu', input_shape=(100, 100, 3)),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # Second Convolutional Layer
    keras.layers.Conv2D(32, (2, 2), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # Two Convolutional Layers
    keras.layers.Conv2D(64, (2, 2), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(64, (2, 2), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),

    keras.layers.Dropout(0.3),  # Dropout to prevent overfitting

    keras.layers.Flatten(),

    # Fully Connected Layer
    keras.layers.Dense(150, activation='relu'),
    keras.layers.Dropout(0.4),

    # Output Layer
    keras.layers.Dense(num_classes, activation='softmax')
])

# Step 4: Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 5: Train the model
history = model.fit(train_generator, epochs=30, validation_data=test_generator)

# Step 6: Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Step 7: Confusion Matrix
y_pred = np.argmax(model.predict(test_generator), axis=1)
y_true = test_generator.classes

conf_matrix = confusion_matrix(y_true, y_pred)

# Plot Confusion Matrix
plt.figure(figsize=(12, 8))
sns.heatmap(conf_matrix, annot=False, fmt="d", cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()
